In [1]:
# Importing Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Setting the Path
path = 'CT_images'

In [3]:
# Loading the images
train_data_dir = os.path.join(path, 'Train')
test_data_dir = os.path.join(path, 'Test')

In [4]:
# Image Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Training the Model
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
    )

Found 3000 images belonging to 2 classes.
Found 900 images belonging to 2 classes.


In [6]:
model = Sequential()

#Adding convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adding a second convolutional layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# Training the Model
model.fit(
    train_generator,
    steps_per_epoch=32,
    epochs=46,
    validation_data=test_generator,
    validation_steps=32)

Epoch 1/46
32/32 [==============================] - 21s 629ms/step - loss: 0.3714 - accuracy: 0.8504 - val_loss: 0.7621 - val_accuracy: 0.5989
Epoch 2/46
32/32 [==============================] - 11s 349ms/step - loss: 0.0437 - accuracy: 0.9873
Epoch 3/46
32/32 [==============================] - 11s 328ms/step - loss: 0.0178 - accuracy: 0.9961
Epoch 4/46
32/32 [==============================] - 10s 314ms/step - loss: 0.0083 - accuracy: 0.9990
Epoch 5/46
32/32 [==============================] - 10s 296ms/step - loss: 0.0039 - accuracy: 0.9990
Epoch 6/46
32/32 [==============================] - 10s 295ms/step - loss: 0.0066 - accuracy: 0.9990
Epoch 7/46
32/32 [==============================] - 9s 293ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 8/46
32/32 [==============================] - 9s 289ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 9/46
32/32 [==============================] - 10s 309ms/step - loss: 0.0025 - accuracy: 0.9990
Epoch 10/46
32/32 [==============================] 

In [9]:
# Saving the Model
model.save('kidney_stone_detection_model.h5')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [11]:
model.evaluate(test_generator)

29/29 [==============================] - 5s 159ms/step - loss: 1.3067 - accuracy: 0.8433


[1.306713342666626, 0.8433333039283752]

# SVM

In [12]:
#Importing the required Modules
import cv2 
import numpy as np 
from skimage.feature import hog 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
import os

In [13]:
#Defining a function to read images from the train and test folders
def read_images(path):
    images_list = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path,filename))
        if img is not None:
            images_list.append(img)
    return images_list

In [14]:
#Reading train images from the normal and stone folders
train_normal = read_images('CT_images/Train/Normal')
train_stone = read_images('CT_images/Train/Stone')

In [15]:
#Creating a list of labels for training 
labels = ['Normal' for item in train_normal] + ['Stone' for item in train_stone]

In [16]:
#Defining a function for HOG feature extraction
def extract_features(images):
    feature_list = []
    for img in images:
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), 
                            cells_per_block=(1, 1), visualize=True, channel_axis=2)
        # Resize the HOG features to a fixed size
        fd = np.resize(fd, (2400, 1))
        # Flatten the array to 2 dimensions
        fd = fd.flatten()
        feature_list.append(fd)
    return feature_list

In [17]:
#Extracting the HOG features from both normal and stone images
feature_list_normal = extract_features(train_normal)
feature_list_stone = extract_features(train_stone)

In [18]:
print(len(feature_list_normal))
print(len(feature_list_stone))

2000
1000


In [19]:
#Combining the features for both classes
features = feature_list_normal + feature_list_stone

In [20]:
#Reading test images from the normal and stone folders
test_normal = read_images('CT_images/Test/Normal')
test_stone = read_images('CT_images/Test/Stone')

In [21]:
#Creating a list of labels for testing 
test_labels = ['Normal' for item in test_normal] + ['Stone' for item in test_stone]

In [22]:
#Creating a Feature Vector for Test Set
test_feature_list_normal = extract_features(test_normal)
test_feature_list_stone = extract_features(test_stone)

In [23]:
print(len(test_feature_list_normal))
print(len(test_feature_list_stone))

600
300


In [24]:
#Combining the features for both classes
test_features = test_feature_list_normal + test_feature_list_stone

In [25]:
#Splitting the data into train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=0)

In [26]:
# Print the shape of the first element in the X_train array
print(X_train[0].shape)

# Print the shape of the second element in the X_train array
print(X_train[1].shape)

# Print the shape of the last element in the X_train array
print(X_train[-1].shape)

(2400,)
(2400,)
(2400,)


In [27]:
# Training a SVM Model
svc = SVC(kernel='rbf', C=1, gamma='auto')
svc.fit(X_train, y_train)

SVC(C=1, gamma='auto')

In [28]:
# Predicting the Test Set
y_pred = svc.predict(X_valid)

In [29]:
#Calculating the accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy : ", accuracy)

Accuracy :  0.7966666666666666


In [34]:
import sklearn.externals
import joblib

# Save the model to a file
joblib.dump(svc, 'svc.pkl')

['svc.pkl']

# GUI

## If you dont want to train the models you can load the models which I have already trained otherwise you can skip these loading models cell and directly run the GUI

In [2]:
from keras.models import load_model
model = load_model('kidney_stone_detection_model.h5')

In [5]:
import sklearn.externals
import joblib
svc = joblib.load("svc.pkl") 

In [30]:
# Function to Select Image
def browse_btn():
    global image_name
    
    label_cnn.configure(text="")
    label.configure(text="")
    
    image_name = askopenfilename(title='Select Image')
    img = Image.open(image_name)
    img = img.resize((200, 200), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    panel = tk.Label(root, image=img)
    panel.image = img
    panel.grid(row=0,column=1,sticky='nw',padx=20,pady=28)

In [31]:
# Function to Predict CNN
def predict_btn_cnn():
    global label_prediction
    global image_name
    test_img = image.load_img(image_name, target_size=(150, 150))
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img, axis=0)
    result = model.predict(test_img)
    if result[0][0] == 1:
        label_cnn.configure(text="Kidney Stone Detected")
    elif result[0][0] == 0:
        label_cnn.configure(text="No Kidney Stone Detected")

In [34]:
#Function for Predict SVM        
def predict_btn_svm():
    global label_prediction
    global image_name
    test_img = cv2.imread(image_name)
    #test_img = image.load_img(image_name, target_size=(150, 150))
    #test_img = image.img_to_array(test_img)
    feature_list_of_img = extract_features([test_img])
    result = svc.predict(feature_list_of_img)    
    #Displaying the output
    if result[0] == 'Stone':
        label.configure(text = "Kidney Stone Detected")
    elif result[0] == 'Normal':
        label.configure(text = "No Kidney Stone Detected")

In [35]:
# Creating the GUI
from tkinter import *
from PIL import Image, ImageTk
import customtkinter
import tkinter
import tkinter as tk
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
from tkinter import messagebox
import keras.utils as image
import numpy as np
import os

customtkinter.set_appearance_mode("System")
root = customtkinter.CTk()

#window size
root.rowconfigure(0,weight=1)    
root.columnconfigure(0,weight=1)

root.geometry('420x380')
root.title('Kidney Stone Detection')
        
# Browse Button
browsebtn = customtkinter.CTkButton(master=root, text="Browse Image", command=browse_btn)
browsebtn.grid(row=0, column=0,sticky='nw',padx=20,pady=20)


# Predict Butoon CNN
predictbtn = customtkinter.CTkButton(master=root, text="Predict CNN", command=predict_btn_cnn)
predictbtn.grid(row=1, column=0,sticky='nw',padx=20,pady=20)

#Label Result CNN
label_cnn = customtkinter.CTkLabel(root, text="")
label_cnn.grid(row=1,column=1,sticky='nw',padx=20,pady=20)

#Label Result SVM
label = customtkinter.CTkLabel(root, text="")
label.grid(row=2,column=1,sticky='nw',padx=20,pady=20)
# Predict Butoon SVM
predictbtnsvm = customtkinter.CTkButton(master=root, text="Predict SVM", command=predict_btn_svm)
predictbtnsvm.grid(row=2, column=0,sticky='nw',padx=20,pady=20)

# Running the GUI
root.mainloop()

C:\Users\Junaid Ali\AppData\Local\Temp\ipykernel_2536\2586483366.py:10: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


1/1 [==============================] - 0s 16ms/step
